<a href="https://colab.research.google.com/github/hilini/Temporary_add_to_version_control/blob/master/Copy_of_cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코랩 메뉴 -> 수정 -> 노트 설정 -> 하드웨어 가속기 GPU 권장(행렬 연산이 많기 때문)

구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


폴더 경로 설정

In [2]:
workspace_path = '/gdrive/My Drive/Colab Notebooks'

필요 패키지 로드

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import time

결과 재현을 위한 설정

In [4]:
seed = 719
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

use_cuda = torch.cuda.is_available()  
device = torch.device("cuda" if use_cuda else "cpu") 
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} 
print("set vars and device done")

set vars and device done


합성곱 신경망(CNN) 정의

In [8]:
import torchvision.models as models

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 100),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x

print("init model done")

init model done


In [9]:
##   AverageMeter 정의
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count



###   학습, 테스트용 함수 정의
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()  
    summary_loss = AverageMeter()  
    summary_acc = AverageMeter()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  
        optimizer.zero_grad() 
        output = model(data)  
        loss = criterion(output, target) 
        loss.backward()  
        optimizer.step() 
        summary_loss.update(loss.detach().item()) 
        pred = output.argmax(dim=1, keepdim=True)  
        correct = pred.eq(target.view_as(pred)).sum().item()
        summary_acc.update(correct / data.size(0)) 
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}, Accuracy: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), summary_loss.avg, summary_acc.avg))
            
    return summary_loss.avg, summary_acc.avg

def test(log_interval, model, device, test_loader):
    model.eval() 
    summary_loss = AverageMeter() 
    summary_acc = AverageMeter() 
    with torch.no_grad():  
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  
            output = model(data) 
            loss = criterion(output, target)  
            summary_loss.update(loss.detach().item())  
            pred = output.argmax(dim=1, keepdim=True)  
            correct = pred.eq(target.view_as(pred)).sum().item() 
            summary_acc.update(correct / data.size(0))  

    print('\nTest set: Average loss: {:.4f}, Accuracy: {:.6f}\n'.format
          (summary_loss.avg, summary_acc.avg))  

    return summary_loss.avg, summary_acc.avg

In [11]:
##   모델 학습을 위한 하이퍼파라미터 셋팅
batch_size = 124
test_batch_size = 1000
max_epochs = 50
lr = 0.01
momentum = 0.9
log_interval = 200

##   데이터 로더 정의 (학습용, 테스트용 따로 정의)
transform_train = transforms.Compose([
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  

transform_test = transforms.Compose([
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  


train_loader = torch.utils.data.DataLoader(
  datasets.CIFAR100(os.path.join(workspace_path, 'data'), train=True, download=True, 
                   transform=transform_train ), 
    batch_size = batch_size, shuffle=True, drop_last=True, **kwargs) 


test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100(os.path.join(workspace_path, 'data'), train=False, download=True,
                         transform=transform_test), 
    batch_size=test_batch_size, shuffle=False, **kwargs)


##   모델, 최적화 알고리즘, 손실 함수 정의
#model = AlexNet().to(device)
model = models.resnet18().to(device)
#model= models.alexnet().to(device)
#model = models.vgg16().to(device)
print(model)

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
criterion = nn.CrossEntropyLoss()


##   학습, 테스트, 모델 저장 수행
best_acc = 0
best_epoch = 0
for epoch in range(1, max_epochs+1):
    start_tic = time.time()
    train_loss, train_acc = train(log_interval, model, device, train_loader, optimizer, epoch)
    test_loss, test_acc = test(log_interval, model, device, test_loader)
   
    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch
        # torch.save(model, os.path.join(workspace_path, f'cifar10_cnn_model_best_acc_{best_epoch}-epoch.pt'))
        # print(f'# save model: cifar10_cnn_model_best_acc_{best_epoch}-epoch.pt\n')
        print('best_acc : ',best_acc,'\n')
    print('epoch time : ', time.time()-start_tic,'\n')

print(f'\n\n# Best accuracy model({best_acc * 100:.2f}%)')

Files already downloaded and verified
Files already downloaded and verified
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1

## baseline보다 성능 높이기
- 예시: 네트워크 구조 개선, 하이퍼파라미터 수정 등

개선 아이디어 설명: 